# 데이터 전처리

### OSRM으로 도보거리 구하기

In [611]:
import os
import numpy as np
import pandas as pd
import requests
from tqdm import tqdm

path = r"C:\Users\82102\OneDrive\바탕 화면\경희대학교\교외활동\공모전\제7회교육부공공데이터분석\optimization\data"
file_demand = "data_optimization_demand.xlsx"
file_candidate = "data_optimization_candidate.xlsx"
file_distance = "data_optimization_distance.xlsx"
file_school = "학교통합정보_cleaned.csv"

df_demand = pd.read_excel(os.path.join(path, file_demand))
df_candidate = pd.read_excel(os.path.join(path, file_candidate))
df_distance = pd.read_excel(os.path.join(path, file_distance))
df_school = pd.read_csv(os.path.join(path, file_school))

In [612]:
# 수요지 정보
df_demand

,지역,학교명,학생수,mid,high,위도,경도
0,김포시,고촌중학교,520,1,0,37.608568,126.768120
1,김포시,김포여자중학교,236,1,0,37.618471,126.713223
2,김포시,금파중학교,1050,1,0,37.624025,126.718912
3,김포시,김포중학교,267,1,0,37.623074,126.707967
4,김포시,대곶중학교,144,1,0,37.649308,126.583731
...,...,...,...,...,...,...,...
67,김포시,솔터고등학교,1094,0,1,37.641773,126.636375
68,김포시,운양고등학교,1092,0,1,37.644970,126.689910
69,김포시,김포제일고등학교,1226,0,1,37.653071,126.677468
70,김포시,고촌고등학교,1001,0,1,37.604586,126.749819


In [613]:
# 후보지 정보
df_candidate

,지역,시설명,예상좌석수,위도,경도,좌석(인구)수,가중치,좌석(인구)수*가중치
0,김포시,김포시고촌도서관,60,37.603305,126.770949,238.0,0.25,59.50
1,김포시,김포시마산도서관,60,37.642428,126.639694,372.0,0.25,93.00
2,김포시,김포시양곡도서관,60,37.661623,126.631437,161.0,0.25,40.25
3,김포시,김포시장기도서관,60,37.641795,126.676544,465.0,0.25,116.25
4,김포시,김포시중봉도서관,60,37.625805,126.706930,0.0,0.25,0.00
...,...,...,...,...,...,...,...,...
124,김포시,솔터고등학교,60,37.641773,126.636375,NaN,NaN,NaN
125,김포시,운양고등학교,60,37.644970,126.689910,NaN,NaN,NaN
126,김포시,김포제일고등학교,60,37.653071,126.677468,NaN,NaN,NaN
127,김포시,고촌고등학교,60,37.604586,126.749819,NaN,NaN,NaN


In [614]:
# 수요지와 후보지 간 도보거리
df_distance

,지역,학교명,시설명,도보거리
0,김포시,고촌중학교,김포시고촌도서관,907.3
1,김포시,고촌중학교,김포시마산도서관,13472.0
2,김포시,고촌중학교,김포시양곡도서관,15496.9
3,김포시,고촌중학교,김포시장기도서관,9901.0
4,김포시,고촌중학교,김포시중봉도서관,6507.6
...,...,...,...,...
3577,포천시,송우고등학교,영북고등학교,33236.0
3578,포천시,송우고등학교,일동고등학교,28309.4
3579,포천시,송우고등학교,포천고등학교,9513.4
3580,포천시,송우고등학교,포천일고등학교,10027.3


# p-median

### p-median 모델링

In [615]:
import pulp

In [616]:
region_input = input("지역을 입력하세요 (예: 포천시): ").strip()
supply_input = int(input("총 수용할 수요량 (예: 100): "))

In [617]:
def capacitated_pmedian(region, supply):
    # 같은 지역 내에서만 최적화
    df_demand_region = df_demand[df_demand['지역'] == region].copy()
    df_candidate_region = df_candidate[df_candidate['지역'] == region].copy()
    df_distance_region = df_distance[df_distance['지역'] == region].copy()
    
    # 수요 가중치 (공급 예정 수/전체 학생 수)
    total_student = df_demand_region["학생수"].sum()
    weight = supply / total_student

    # 수요지별 수요량
    demand_dict = {}
    for idx, row in df_demand_region.iterrows():
        key = (row['지역'], row['학교명'])
        demand_dict[key] = round(row['학생수'] * weight, 0)
    
    # 후보지의 용량
    capacity_dict = {}
    for idx, row in df_candidate_region.iterrows():
        key = (row['지역'], row['시설명'])
        capacity_dict[key] = row['예상좌석수']
        
    # 수요지와 후보지 간 도보거리
    distance_dict = {}
    for idx, row in df_distance_region.iterrows():
        key_demand = (row['지역'], row['학교명'])
        key_candidate = (row['지역'], row['시설명'])
        distance_dict[(key_demand, key_candidate)] = row['도보거리']
        
    # 최종 선정된 시설과 각 시설에 할당된 수요지 및 할당량
    selected_facility = []
    assignment = {}
    
    # 선정되지 않은 후보지
    remaining_candidate = set(capacity_dict.keys())

    # 할당된 수요량의 총합
    total_assigned_demand = 0  

    # 선정 가능한 후보지 도출
    for iter in range(50):
        # 후보지에 할당된 수요량의 총합 > 용량인 경우, 중단
        if total_assigned_demand > supply:
            break

        # 모든 수요지의 수요량이 충족된 경우, 중단
        available_demand = [node for node, demand in demand_dict.items() if demand > 0]
        if not available_demand:
            break
        
        # 아직 선정되지 않은 후보지
        available_candidate = [cand for cand in remaining_candidate if cand not in selected_facility]

        # 해당 반복에서 조건을 만족하는 후보지
        eligible_candidate = []
        
        for candidate in available_candidate:
            # 후보지와의 거리가 17308m 이하인 수요지들 선택
            demand_in_range = [i for i in available_demand 
                                if (i, candidate) in distance_dict and distance_dict[(i, candidate)] <= 17308]
            if not demand_in_range:
                continue
            
            # 17308m 이내 수요지들을 가까운 순으로 정렬
            demand_sorted = sorted(demand_in_range, key=lambda i: distance_dict[(i, candidate)])

            # 각 후보지에 수요지 및 수요량 할당
            # 최초로 후보지 용량을 초과하는 경우, 남은 용량만 할당
            cumulative_demand = 0
            allocated_school = []

            for school in demand_sorted:
                demand = demand_dict[school]
                # 해당 수요지의 수요량 <= 남은 용량인 경우, 전체 할당
                if cumulative_demand + demand <= capacity_dict[candidate]:
                    allocated_school.append((school, demand))
                    cumulative_demand += demand
                # 해당 수요지의 수요량 > 남은 용량인 경우, 부분 할당
                elif cumulative_demand < capacity_dict[candidate]:
                    remaining_capacity = capacity_dict[candidate] - cumulative_demand
                    allocated_school.append((school, remaining_capacity))
                    cumulative_demand += remaining_capacity
                    break
                else:
                    break
            
            # 해당 후보지에 할당된 수요량 < 용량의 0.8배인 경우, 다음 후보지로
            if cumulative_demand < 0.8 * capacity_dict[candidate]:
                continue
            
            # 목적함수: 할당된 수요량 x 수요지까지의 거리
            cost = sum(allocated_demand * distance_dict[(school, candidate)] 
                       for (school, allocated_demand) in allocated_school)
            
            eligible_candidate.append((candidate, allocated_school, cumulative_demand, cost))
        
        if not eligible_candidate:
            break
        
        # 목적함수 비용이 최소인 후보지를 선정
        chosen_facility, allocated_school, cumulative_demand, cost = min(eligible_candidate, key=lambda x: x[3])
        selected_facility.append(chosen_facility)
        
        # 선정된 시설에 수요지 할당
        # 후보지와의 거리가 17308m 이하인 수요지들 선택
        demand_in_range = [i for i in available_demand 
                            if (i, chosen_facility) in distance_dict and distance_dict[(i, chosen_facility)] <= 17308]
        # 17308m 이내 수요지들을 가까운 순으로 정렬
        demand_sorted = sorted(demand_in_range, key=lambda i: distance_dict[(i, chosen_facility)])
        # 각 후보지에 수요지 및 수요량 할당
        remaining_capacity = capacity_dict[chosen_facility]
        allocated_school = []
        for school in demand_sorted:
            if remaining_capacity <= 0:
                break
            demand = demand_dict[school]
            if demand <= 0:
                continue
            allocation = min(demand, remaining_capacity)
            allocated_school.append((school, allocation))
            demand_dict[school] -= allocation
            remaining_capacity -= allocation
            total_assigned_demand += allocation

        assignment[chosen_facility] = allocated_school
        print(f"Iteration {iter+1}: 시설 = {chosen_facility}, 학교 = {allocated_school}")
        
        # 할당 후, 만약 어떤 학교의 남은 수요가 0이면,
        # 후보 시설 중 해당 학교명과 동일한 시설은 remaining_candidate에서 제거
        for (school, allocation) in allocated_school:
            if demand_dict[school] == 0:
                school_name = school[1]
                to_remove = [cand for cand in remaining_candidate if cand[1] == school_name]
                for cand in to_remove:
                    remaining_candidate.remove(cand)
                    
    # 반복 종료 후, 아직 미할당된 수요가 있는 학교 목록 출력
    not_covered_schools = [node for node, demand in demand_dict.items() if demand > 0]
    print("충족하지 못한 학교:", not_covered_schools)
    
    return selected_facility, assignment, not_covered_schools, total_assigned_demand



### p-median 시각화

In [618]:
import folium
from matplotlib import cm, colors as mcolors

def map_pmedian(region, selected_facility, assignment):
    """
    region: str, ex) '영통구'
    selected_facility: List of tuples [(region, facility_name), ...] (순서대로)
    assignment: dict mapping (region, facility_name) -> [ ((region, school_name), allocated_amount), ... ]
    osm_opacity: float 0.0~1.0, 배경 OSM 타일의 불투명도
    """
    # 1) 지역별 데이터 필터링
    df_fac = df_candidate[df_candidate['지역'] == region].copy()
    df_sch = df_demand   [df_demand   ['지역'] == region].copy()

    # 2) 지도 초기화 (기본 타일 꺼두기)
    center_lat = df_fac['위도'].mean()
    center_lon = df_fac['경도'].mean()
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles=None,           # 기본 OSM 타일 비활성화
        control_scale=True
    )

    # 3) OSM 타일레이어 재추가 + 불투명도 설정
    folium.TileLayer(
        'OpenStreetMap',
        name='OSM (faded)',
        opacity=0.8,  # 0.0~1.0 사이에서 조절
        control=False
    ).add_to(m)

    # 4) iteration별 색상 통일 (tab10 10색)
    base10 = cm.get_cmap('tab10').colors
    hex10  = [mcolors.to_hex(c) for c in base10]
    iter_colors = {
        i: hex10[i % len(hex10)]
        for i in range(len(selected_facility))
    }
    # 5) p-median 결과 시각화
    for i, (reg, fac_name) in enumerate(selected_facility):
        # 5-1) 시설 좌표
        fac_row = df_fac[df_fac['시설명'] == fac_name]
        if fac_row.empty: continue
        lat, lon = fac_row.iloc[0][['위도','경도']]
        color    = iter_colors[i]

        # 5-2) DivIcon: flexbox 중앙 정렬 + anchor 설정
        html = f"""
        <div style="
            display: flex;
            justify-content: center;
            align-items: center;
            background-color: {color};
            color: white;
            border-radius: 50%;
            width: 28px;
            height: 28px;
            font-size: 12pt;
            font-weight: bold;
            box-shadow: 0 0 3px #555;">
            {i+1}
        </div>"""
        folium.Marker(
            location=(lat, lon),
            icon=folium.DivIcon(
                html=html,
                icon_size=(28, 28),
                icon_anchor=(14, 14)
            ),
            popup=folium.Popup(f"{fac_name} (#{i+1})", max_width=150),
            tooltip=fac_name
        ).add_to(m)

        # 5-3) 할당된 학교(수요지) CircleMarker 및 선 연결
        key = (reg, fac_name)
        if key in assignment:
            for (sch_key, alloc) in assignment[key]:
                sch_name = sch_key[1]
                sch_row  = df_sch[df_sch['학교명'] == sch_name]
                if sch_row.empty: continue
                s_lat, s_lon = sch_row.iloc[0][['위도','경도']]

                # 학교 마커
                folium.CircleMarker(
                    location=(s_lat, s_lon),
                    radius=6,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.8,
                    popup=folium.Popup(f"{sch_name}\n할당량: {alloc}", max_width=150),
                    tooltip=sch_name
                ).add_to(m)

                # 시설 ↔ 학교 PolyLine
                folium.PolyLine(
                    locations=[(lat, lon), (s_lat, s_lon)],
                    color=color,
                    weight=2,
                    opacity=0.7
                ).add_to(m)

    # (선택) 레이어 토글
    # folium.LayerControl().add_to(m)

    return m


### p-median 결과

In [619]:
# 최적화 실행 및 결과 출력
selected_facility, assignment, not_covered_schools, cumulative_demand = capacitated_pmedian(region_input, supply_input)
print("실제로 공급 받는 총 학생 수:", cumulative_demand)

# 시각화 실행 및 결과 출력
m = map_pmedian(region_input, selected_facility, assignment)
display(m)   

Iteration 1: 시설 = ('포천시', '동남고등학교'), 학교 = [(('포천시', '동남고등학교'), np.float64(33.0)), (('포천시', '동남중학교'), np.float64(18.0)), (('포천시', '대경중학교'), np.float64(9.0))]
Iteration 2: 시설 = ('포천시', '송우고등학교'), 학교 = [(('포천시', '송우고등학교'), np.float64(35.0)), (('포천시', '대경중학교'), np.float64(8.0)), (('포천시', '송우중학교'), np.float64(17.0))]
Iteration 3: 시설 = ('포천시', '포천일고등학교'), 학교 = [(('포천시', '포천일고등학교'), np.float64(29.0)), (('포천시', '포천여자중학교'), np.float64(19.0)), (('포천시', '포천고등학교'), np.float64(12.0))]
Iteration 4: 시설 = ('포천시', '포천고등학교'), 학교 = [(('포천시', '포천고등학교'), np.float64(18.0)), (('포천시', '포천중학교'), np.float64(22.0)), (('포천시', '경북중학교'), np.float64(3.0)), (('포천시', '삼성중학교'), np.float64(1.0)), (('포천시', '송우중학교'), np.float64(9.0)), (('포천시', '갈월중학교'), np.float64(7.0))]
Iteration 5: 시설 = ('포천시', '이동면행정복지센터'), 학교 = [(('포천시', '이동중학교'), np.float64(4.0)), (('포천시', '일동중학교'), np.float64(14.0)), (('포천시', '일동고등학교'), np.float64(14.0)), (('포천시', '영북고등학교'), np.float64(8.0)), (('포천시', '영중중학교'), np.float64(4.0)), (('포천시', '영북중학교'), n

C:\Users\82102\AppData\Local\Temp\ipykernel_14232\2613558629.py:34: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  base10 = cm.get_cmap('tab10').colors


# MCLP

### MCLP 모델링

In [620]:
region_input = input("지역을 입력하세요 (예: 포천시): ").strip()
supply_input = int(input("총 수용할 수요량 (예: 100): "))

In [621]:
def capacitated_MCLP(region, coverage_threshold, supply):
    # 같은 지역 내에서만 최적화
    df_demand_region = df_demand[df_demand['지역'] == region].copy()
    df_candidate_region = df_candidate[df_candidate['지역'] == region].copy()
    df_distance_region = df_distance[df_distance['지역'] == region].copy()

    # 수요 가중치 (공급 예정 수/전체 학생 수)
    total_students = df_demand_region["학생수"].sum()
    weight = supply / total_students

    # 수요지별 수요량
    demand_dict = {}
    for idx, row in df_demand_region.iterrows():
        key = (row['지역'], row['학교명'])
        demand_dict[key] = round(row['학생수'] * weight, 0)

    # 후보지의 용량
    capacity_dict = {}
    for idx, row in df_candidate_region.iterrows():
        key = (row['지역'], row['시설명'])
        capacity_dict[key] = row['예상좌석수']

    # 수요지와 후보지 간 도보거리
    distance_dict = {}
    for idx, row in df_distance_region.iterrows():
        key_demand = (row['지역'], row['학교명'])
        key_candidate = (row['지역'], row['시설명'])
        distance_dict[(key_demand, key_candidate)] = row['도보거리']

    # 동률 상황에서 후보지 우선순위용 가중치 정보
    candidate_weight = {}
    for idx, row in df_candidate_region.iterrows():
        key = (row['지역'], row['시설명'])
        candidate_weight[key] = row['좌석(인구)수*가중치']

    # 최종 선정된 시설과 각 시설에 할당된 수요지 및 할당량
    selected_facility = []  
    assignment = {}      

    # 할당된 수요량의 총합
    total_assigned_demand = 0 

    # 선정 가능한 후보지 도출
    for iteration in range(50):
        # 후보지에 할당된 수요량의 총합 > 용량인 경우, 중단
        if total_assigned_demand > supply:
            break

        # 모든 수요지의 수요량이 충족된 경우, 중단
        available_demand = {school: demand for school, demand in demand_dict.items() if demand > 0}
        if not available_demand:
            break

        # 후보지별 할당 가능한 수요와 커버리지 내 총 수요량
        candidate_values = {}  
        candidate_assignment = {}
        
        for candidate, capacity in capacity_dict.items():
            # 아직 선정되지 않은 후보지에 대하여
            if candidate in [fac for (_, fac) in selected_facility]:
                continue

            # 해당 후보지의 커버리지 내 수요지
            eligible_schools = []
            for school, demand in available_demand.items():
                if (school, candidate) in distance_dict and distance_dict[(school, candidate)] <= coverage_threshold:
                    eligible_schools.append((distance_dict[(school, candidate)], school))
            if not eligible_schools:
                continue
            
            # 수요지들을 가까운 순으로 정렬
            eligible_schools.sort(key=lambda x: x[0])

            # 커버리지 내 총 수요량 < 용량의 0.8배 이하인 경우, 다음 후보지로
            total_possible_demand = sum(available_demand[school] for _, school in eligible_schools)
            if total_possible_demand < capacity*0.8:
                break
            
            # 각 후보지에 수요지 및 수요량 할당
            remaining_capacity = capacity
            assigned_demand = 0
            assignment_list = [] 

            for _, school in eligible_schools:
                if remaining_capacity <= 0:
                    break
                school_demand = available_demand[school]
                allocation = min(school_demand, remaining_capacity, supply - total_assigned_demand)
                if allocation <= 0:
                    break
                assignment_list.append((school, allocation))
                remaining_capacity -= allocation
                assigned_demand += allocation

            candidate_values[candidate] = (assigned_demand, total_possible_demand)
            candidate_assignment[candidate] = assignment_list

        # 할당 가능한 후보지가 없으면 종료
        if not candidate_values:
            break

        # 시설 선택 기준

        # (1) 할당된 수요량이 가장 큰 후보지
        max_assigned_demand = max(val[0] for val in candidate_values.values())
        best_candidates = [candidate for candidate, (ad, _) in candidate_values.items() if ad == max_assigned_demand]

        # (2) 후보지가 여러 개인 경우, 커버리지 내 총 수요가 가장 큰 후보지
        if len(best_candidates) > 1:
            max_total_possible = max(candidate_values[candidate][1] for candidate in best_candidates)
            best_candidates = [candidate for candidate in best_candidates if candidate_values[candidate][1] == max_total_possible]

        # (3) 후보지가 여러 개인 경우, 시설명이 "학교"로 끝나지 않는 후보지
        if len(best_candidates) > 1:
            non_school_candidates = [candidate for candidate in best_candidates if not candidate[1].endswith("학교")]
            if non_school_candidates:
                best_candidates = non_school_candidates

        # (4) 후보지가 여러 개인 경우, "좌석(인구)수*가중치" 값이 가장 높은 후보지
        if len(best_candidates) > 1:
            best_candidates.sort(key=lambda c: candidate_weight.get(c, 0), reverse=True)
            best_candidates = [best_candidates[0]]

        # (5) 후보지가 여러 개인 경우, 해당 시설과 이름이 같은 수요지의 수요량이 가장 큰 후보지
        if len(best_candidates) > 1:
            max_same_name_demand = -1
            selected = []
            for candidate in best_candidates:
                region_name, facility_name = candidate
                same_name_demand = demand_dict.get((region_name, facility_name), 0)
                if same_name_demand > max_same_name_demand:
                    max_same_name_demand = same_name_demand
                    selected = [candidate]
                elif same_name_demand == max_same_name_demand:
                    selected.append(candidate)
            best_candidates = selected

        for candidate in best_candidates:
            selected_facility.append((iteration+1, candidate))
            assignment[candidate] = candidate_assignment[candidate]
            print(f"Iteration {iteration+1}: 시설 = {candidate}, 학교 = {candidate_assignment[candidate]}")
            
            for school, assigned_amt in candidate_assignment[candidate]:
                demand_dict[school] = max(0, demand_dict[school] - assigned_amt)
                total_assigned_demand += assigned_amt

        iteration += 1

    # 아직 미할당된 수요가 있는 학교 목록 출력
    not_covered_schools = [school for school, demand in demand_dict.items() if demand > 0]
    print("충족하지 못한 학교:", not_covered_schools)

    return selected_facility, assignment, not_covered_schools, total_assigned_demand



### MCLP 시각화

In [622]:
import folium
from matplotlib import cm, colors as mcolors

def map_MCLP(region, selected_facility, assignment):
    # 1) 지역 필터링
    df_fac = df_candidate[df_candidate['지역'] == region].copy()
    df_sch = df_demand   [df_demand   ['지역'] == region].copy()

    # 2) 지도 초기화 (기본 타일 끄기)
    center_lat = df_fac['위도'].mean()
    center_lon = df_fac['경도'].mean()
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles=None,           # 기본 OSM 비활성화
        control_scale=True
    )

    # 3) OSM 타일레이어 재추가 + 불투명도 설정
    folium.TileLayer(
        'OpenStreetMap',
        name='OSM (faded)',
        opacity=0.8,  # 0.0(완전투명) ~ 1.0(불투명)
        control=False
    ).add_to(m)

    # 4) iteration별 색상 통일 (tab10 10색)
    base10 = cm.get_cmap('tab10').colors
    hex10  = [mcolors.to_hex(c) for c in base10]
    iteration_list = sorted({iter_num for iter_num, _ in selected_facility})
    iter_to_color = {
        iter_num: hex10[i % len(hex10)]
        for i, iter_num in enumerate(iteration_list)
    }

    # 5) 시설 + 할당 학교 시각화
    for iter_num, (region_name, fac_name) in selected_facility:
        fac_row = df_fac[df_fac['시설명'] == fac_name]
        if fac_row.empty:
            continue
        fac_lat = fac_row.iloc[0]['위도']
        fac_lon = fac_row.iloc[0]['경도']
        color   = iter_to_color[iter_num]

        # (1) DivIcon: 색 배경 + iteration 숫자
        html = f"""
        <div style="
            background-color: {color};
            color: white;
            border-radius: 50%;
            text-align: center;
            font-size: 10pt;
            font-weight: bold;
            width: 24px;
            height: 24px;
            line-height: 24px;
            box-shadow: 0 0 3px #555;">
            {iter_num}
        </div>"""
        folium.Marker(
            location=(fac_lat, fac_lon),
            icon=folium.DivIcon(html=html),
            popup=f"{fac_name} (Iter {iter_num})",
            tooltip=f"{fac_name} (Iter {iter_num})"
        ).add_to(m)

        # (2) 할당된 학교 + 선 연결
        key = (region_name, fac_name)
        if key in assignment:
            for (school_region, sch_name), _ in assignment[key]:
                sch_row = df_sch[df_sch['학교명'] == sch_name]
                if sch_row.empty:
                    continue
                sch_lat = sch_row.iloc[0]['위도']
                sch_lon = sch_row.iloc[0]['경도']

                folium.CircleMarker(
                    location=(sch_lat, sch_lon),
                    radius=5,
                    color=color,
                    fill=True,
                    fill_color=color,
                    fill_opacity=0.7,
                    popup=sch_name,
                    tooltip=sch_name
                ).add_to(m)

                folium.PolyLine(
                    locations=[(fac_lat, fac_lon), (sch_lat, sch_lon)],
                    color=color,
                    weight=2,
                    opacity=0.7
                ).add_to(m)

    return m


### MCLP 결과

In [623]:
coverage_threshold = 5000
 
# 최적화 실행 및 결과 출력
selected_facility, assignment, not_covered_schools, cumulative_demand = capacitated_MCLP(region_input, coverage_threshold, supply_input)
print("실제로 공급 받는 총 학생 수:", cumulative_demand)

# 시각화 실행 및 결과 출력
result_map = map_MCLP(region_input, selected_facility, assignment)
display(result_map)

Iteration 1: 시설 = ('김포시', '김포시장기도서관'), 학교 = [(('김포시', '운양중학교'), np.float64(12.0)), (('김포시', '고창중학교'), np.float64(10.0)), (('김포시', '운양고등학교'), np.float64(11.0)), (('김포시', '김포제일고등학교'), np.float64(12.0)), (('김포시', '장기중학교'), np.float64(13.0)), (('김포시', '장기고등학교'), np.float64(2.0))]
Iteration 2: 시설 = ('김포시', '김포시중봉도서관'), 학교 = [(('김포시', '김포중학교'), np.float64(3.0)), (('김포시', '김포과학기술고등학교'), np.float64(5.0)), (('김포시', '감정중학교'), np.float64(8.0)), (('김포시', '김포여자중학교'), np.float64(2.0)), (('김포시', '김포고등학교'), np.float64(11.0)), (('김포시', '금파중학교'), np.float64(11.0)), (('김포시', '사우고등학교'), np.float64(12.0)), (('김포시', '풍무고등학교'), np.float64(8.0))]
Iteration 3: 시설 = ('김포시', '김포시양곡도서관'), 학교 = [(('김포시', '양곡고등학교'), np.float64(8.0)), (('김포시', '양곡중학교'), np.float64(4.0)), (('김포시', '신양중학교'), np.float64(3.0)), (('김포시', '김포한가람중학교'), np.float64(9.0)), (('김포시', '솔터고등학교'), np.float64(11.0)), (('김포시', '나래중학교'), np.float64(10.0)), (('김포시', '은여울중학교'), np.float64(10.0)), (('김포시', '운유고등학교'), np.float64(5.0))]
Iteration 4: 시설 = 

C:\Users\82102\AppData\Local\Temp\ipykernel_14232\2389965607.py:28: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  base10 = cm.get_cmap('tab10').colors
